In [1]:
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
from ahyper import utils

%matplotlib inline

In [2]:
with open('data/enron_hypergraph_annotated.json') as file:
    data = json.load(file)

role_fields = ['cc', 'from', 'to']

# add unique ID to each edge:

for i in range(len(data)):
    data[i]['eid'] = -(i+1)

In [4]:
IL = utils.incidence_list_from_records(data, role_fields = role_fields)

records = utils.records_from_incidence_list(IL, role_fields = role_fields)

In [6]:
# quick test that these two functions are inverses: 
records == data

True